In [ ]:
# default_exp connectomes

Module connectomes
---

Pipeline making use of `postprocessing` nodes to generate a connectivity matrix and distance matrix.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#exporti
import os
from pathlib import Path
from nipype import IdentityInterface
from nipype.pipeline import Node, Workflow
from nipype.interfaces.io import SelectFiles, DataSink

import pipetography.nodes as nodes

In [ ]:
# export

class connectome:
    """
    Create a workflow that produces connectomes based on input atlases and streamlines
    
    Inputs:
    atlas_dir (str): base directory of folder containing atlases
    BIDS_dir (str): base BIDS directory path
    atlas_list (List of strings): names of atlases: aal, brainnectome, desikan-killiany, default is set to brainnectome for now.
    """
    
    def __init__(self, atlas_dir, BIDS_dir, atlas_list=['brainnectome'], skip_tuples=[()]):
        """
        Initialize workflow nodes
        """
        self.BIDS = BIDS_dir
        data_dir = os.path.join(Path(BIDS_dir).parent)
        self.atlas_dir = atlas_dir
        self.atlas_list = atlas_list
        self.skip_combos = skip_tuples
        self.atlas_template = {
            "atlases": os.path.join(atlas_dir, "{atlas_name}", "*_1mm.nii.gz")
        }
        self.subject_template = {
            'tck': os.path.join(data_dir, 'cuda_tracking', '_session_id_{session_id}_subject_id_{subject_id}',  'sub-{subject_id}_ses-{session_id}.tck'),
            'dwi': os.path.join(data_dir, 'derivatives', 'preproc_mni', '_session_id_{session_id}_subject_id_{subject_id}', 'dwi_acpc_1mm.nii.gz'),
            'T1A': os.path.join(data_dir, 'derivatives', 't1_acpc_aligned', '_session_id_{session_id}_subject_id_{subject_id}', 'acpc_t1.nii'),
            'mask': os.path.join(data_dir, 'derivatives', 'preproc_mni', '_session_id_{session_id}_subject_id_{subject_id}', 'dwi_acpc_1mm_mask.nii.gz'),
            'mr5tt': os.path.join(data_dir, 'derivatives', 'wm_mask', '_session_id_{session_id}_subject_id_{subject_id}', '5tt_wm.nii.gz')
        }
        
        
    def create_nodes(self):
        """
        Create postprocessing nodes
        """
        self.PostProcNodes = nodes.PostProcNodes(atlas_dir=self.atlas_dir, atlas_list=self.atlas_list, atlas_template = self.atlas_template, BIDS_dir=self.BIDS, subj_template = self.subject_template, skip_tuples = self.skip_combos)
        self.PostProcNodes.set_inputs()
        self.workflow = None
        
        
    def connect_nodes(self, wf_name="connectomes"):
        """
        Connect postprocessing nodes into workflow
        """
        self.workflow = Workflow(name=wf_name, base_dir=os.path.join(Path(self.BIDS).parent, 'derivatives'))
        self.workflow.connect(
            [
                (self.PostProcNodes.subject_source, self.PostProcNodes.select_files, [('subject_id', 'subject_id'),
                                                                                      ('session_id', 'session_id')]),
                (self.PostProcNodes.atlas_source, self.PostProcNodes.select_atlases, [('atlas_name', 'atlas_name')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.linear_reg, [('dwi', 'fixed_image')]),
                (self.PostProcNodes.select_atlases, self.PostProcNodes.linear_reg, [('atlases', 'moving_image')]),
                (self.PostProcNodes.linear_reg, self.PostProcNodes.nonlinear_reg, [('warped_image', 'moving_image')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.nonlinear_reg, [('dwi', 'fixed_image')]),
                (self.PostProcNodes.nonlinear_reg, self.PostProcNodes.round_atlas, [('warped_image', 'in_file')]),
                (self.PostProcNodes.round_atlas, self.PostProcNodes.connectome, [('out_file', 'in_parc')]),
                (self.PostProcNodes.round_atlas, self.PostProcNodes.distance, [('out_file', 'in_parc')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.response, [('dwi', 'in_file')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.fod, [('dwi', 'in_file')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.fod, [('mask', 'mask_file')]),
                (self.PostProcNodes.response, self.PostProcNodes.fod, [('wm_file', 'wm_txt')]),
                (self.PostProcNodes.response, self.PostProcNodes.fod, [('gm_file', 'gm_txt')]),
                (self.PostProcNodes.response, self.PostProcNodes.fod, [('csf_file', 'csf_txt')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.sift2, [('mr5tt', 'act')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.sift2, [('tck', 'in_file')]),
                (self.PostProcNodes.fod, self.PostProcNodes.sift2, [('wm_odf', 'in_fod')]),
                (self.PostProcNodes.sift2, self.PostProcNodes.connectome, [('out_file', 'in_weights')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.connectome, [('tck', 'in_file')]),
                (self.PostProcNodes.select_files, self.PostProcNodes.distance, [('tck', 'in_file')]),
                (self.PostProcNodes.connectome, self.PostProcNodes.datasink, [('out_file', 'matrices.@connectome')]),
                (self.PostProcNodes.distance, self.PostProcNodes.datasink, [('out_file', 'matrices.@distance')])
            ])
        
    def draw_pipeline(self, graph_type='orig'):
        """
        Visualize workflow
        """
        self.workflow.write_graph(graph2use=graph_type, dotfilename='postprocess.dot')
    
    def run_pipeline(self, parallel=None):
        """
        Run nipype workflow
        """
        if type(parallel) == int:
            print("Running workflow with {} parallel processes".format(parallel))
            self.workflow.run('MultiProc', plugin_args = {'n_procs': parallel})
        elif parallel is None:
            print("Parallel processing disabled, running workflow serially")
            self.workflow.run()

## Example

Workflow creating a connectivity matrix and a distance adjacency matrix for brainnectome and desikan-killiany atlas.

In [ ]:
#example
post_wf = connectome(atlas_dir='/Users/xxie/lab/Human_Brain_Atlases', BIDS_dir='/Users/xxie/sample_data/dwipreproc/BIDS_output', atlas_list=['brainnectome', 'desikan-killiany'], skip_tuples=[('01', '003')])

Take a look at the post-processing workflow:

In [ ]:
#example
from IPython.display import Image

post_wf.create_nodes()
post_wf.connect_nodes(wf_name='connectomes')
post_wf.draw_pipeline(graph_type='flat')
Image('/Users/xxie/sample_data/dwipreproc/derivatives/connectomes/postprocess.png')

In [ ]:
#example
post_wf.run_pipeline(parallel=2)